### Basis MuJoCo usage using `mjd_transitionFD`

In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.6]


### Parse `scene_ur5e_rg2_with_sensor.xml`

In [2]:
xml_path = '../asset/ur5e/scene_ur5e_rg2_with_sensor.xml'
env = MuJoCoParserClass(name='UR5e',rel_xml_path=xml_path,VERBOSE=True)
env.reset()

dt:[0.0020] HZ:[500]
n_dof (=nv):[12]
n_geom:[41]
geom_names:['floor', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
n_body:[18]
body_names:['world', 'base', 'shoulder_link', 'upper_arm_link', 'forearm_link', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link', 'tcp_link', 'camera_mount', 'rg2_gripper_base_link', 'camera_center', 'rg2_gripper_finger1_finger_link', 'rg2_gripper_finger1_inner_knuckle_link', 'rg2_gripper_finger1_finger_tip_link', 'rg2_gripper_finger2_finger_link', 'rg2_gripper_finger2_inner_knuckle_link', 'rg2_gripper_finger2_finger_tip_link']
n_joint:[12]
joint_names:['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint', 'gripper_finger1_joint', 'gripper_finger1_inner_knuckle_joint', 'gripper_finger1_finger_tip_joint', 'gripper_f

### Set sensor callback function

In [3]:
def sensor_callback(model,data,stage):
    """
        Sensor callback function
    """
    # Get target
    target_1 = model.sensor('ee_pos_err_1').user
    target_2 = model.sensor('ee_pos_err_2').user
    
    # Get position
    pos_1 = data.body('tcp_link').xpos.copy()
    pos_2 = data.body('tcp_link').xpos.copy()
    
    # Append
    data.sensordata[0] = np.linalg.norm(target_1-pos_1)
    data.sensordata[1] = np.linalg.norm(target_2-pos_2)
    
    # Print
    VERBOSE = False
    if VERBOSE:
        print ("target_1:%s"%(target_1))
        print ("target_2:%s"%(target_2))
        print ("pos_1:%s"%(pos_1))
        print ("pos_2:%s"%(pos_2))

mujoco.set_mjcb_sensor(sensor_callback)

In [4]:
dim_state  = env.model.nv + env.model.nv + env.model.na # qpos, qvel, na? 
dim_sensor = env.model.nsensordata
dim_action = env.model.nu
print (dim_state,dim_sensor,dim_action)

24 2 7


In [6]:
env.init_viewer(viewer_title='UR5e with RG2',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=174.08,distance=2.76,elevation=-33,lookat=[0.1,0.05,0.16],
                  VIS_TRANSPARENT=True,VIS_CONTACTPOINT=True,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,
                  jointrgba=[0.2,0.6,0.8,0.6])
env.reset()

# Get initial joint information
joint_names = [
    'shoulder_pan_joint','shoulder_lift_joint','elbow_joint',
    'wrist_1_joint','wrist_2_joint','wrist_3_joint']
idxs_fwd = env.get_idxs_fwd(joint_names)
q = env.get_qpos_joints(joint_names)

# Set target position
p_trgt = np.array([0.1,0.4,0.98])

while env.is_viewer_alive():
    
    # Set target
    env.model.sensor('ee_pos_err_1').user = p_trgt
    
    # Run 'mjd_transitionFD' to solve IK!
    A = np.zeros((dim_state,dim_state),dtype=np.double)
    B = np.zeros((dim_state,dim_action),dtype=np.double)
    C = np.zeros((dim_sensor,dim_state),dtype=np.double)
    D = np.zeros((dim_sensor,dim_action),dtype=np.double)
    fd_tol  = 0.1 # 1e-8
    fd_mode = 2   # 0: one-sided, 1: centered
    mujoco.mjd_transitionFD(
        env.model,
        env.data,
        fd_tol,
        fd_mode,
        A, # dx'/dx
        B, # dx'/du
        C, # ds/dx
        D, # ds/du
        )
    
    # Probe
    VERBOSE = False
    if VERBOSE:
        print (env.data.sensordata)
        print (C)
        print (D)
    
    # Step
    q = q - 0.01*C[0,:6]
    env.forward(q=q,joint_idxs=idxs_fwd)
    
    # Render
    env.plot_T(p=np.zeros(3),R=np.eye(3,3),PLOT_AXIS=True,axis_len=1.0,axis_width=0.01)
    env.plot_sphere(p=p_trgt,r=0.05,rgba=[1,0,0,0.1])
    env.plot_contact_info()
    env.render(render_every=5)
env.close_viewer()
print ("Done.")

Pressed ESC
Quitting.
Done.
